In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [5]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [6]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.2,
                                         rotation_range=15,
                                         height_shift_range=0.2,
                                         width_shift_range=0.2,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [7]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=x)

vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [8]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 global_average_pooling2d_1 True
20 dense_1 True


In [9]:
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_fc.h5')

Epoch 1/10
156/156 [==============================] - 505s 3s/step - loss: 3.0131 - acc: 0.2836 - val_loss: 2.8905 - val_acc: 0.3615
Epoch 2/10
156/156 [==============================] - 349s 2s/step - loss: 1.2945 - acc: 0.5804 - val_loss: 2.0605 - val_acc: 0.4508
Epoch 3/10
156/156 [==============================] - 341s 2s/step - loss: 0.8740 - acc: 0.7088 - val_loss: 1.7904 - val_acc: 0.5226
Epoch 4/10
156/156 [==============================] - 340s 2s/step - loss: 0.7142 - acc: 0.7590 - val_loss: 1.8088 - val_acc: 0.5340
Epoch 5/10
156/156 [==============================] - 339s 2s/step - loss: 0.5884 - acc: 0.8029 - val_loss: 1.8504 - val_acc: 0.4987
Epoch 6/10
156/156 [==============================] - 339s 2s/step - loss: 0.5324 - acc: 0.8241 - val_loss: 1.7418 - val_acc: 0.5427
Epoch 7/10
156/156 [==============================] - 338s 2s/step - loss: 0.4671 - acc: 0.8482 - val_loss: 1.9800 - val_acc: 0.4869
Epoch 8/10
156/156 [==============================] - 335s 2s/step - 

In [11]:
vgg16_model = load_model('vgg16_GAP_fc.h5')

for layer in vgg16_model.layers[:15]:
    layer.trainable = False
    
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                     optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=3)])

vgg16_model.save('vgg16_GAP_15.h5')

Epoch 1/20
156/156 [==============================] - 345s 2s/step - loss: 0.3773 - acc: 0.8920 - val_loss: 1.1608 - val_acc: 0.6936
Epoch 2/20
156/156 [==============================] - 342s 2s/step - loss: 0.0716 - acc: 0.9803 - val_loss: 1.1697 - val_acc: 0.6995
Epoch 3/20
156/156 [==============================] - 342s 2s/step - loss: 0.0385 - acc: 0.9895 - val_loss: 1.7797 - val_acc: 0.6394
Epoch 4/20
 81/156 [==============>...............] - ETA: 2:31 - loss: 0.0349 - acc: 0.9883

KeyboardInterrupt: 

In [15]:
vgg16_model = load_model('vgg16_GAP_fc_15.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy']) 
    
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=10)])

vgg16_model.save('vgg16_GAP_fc_11.h5')

Epoch 1/50
156/156 [==============================] - 363s 2s/step - loss: 0.0482 - acc: 0.9849 - val_loss: 1.2905 - val_acc: 0.7014
Epoch 2/50
156/156 [==============================] - 352s 2s/step - loss: 0.0411 - acc: 0.9876 - val_loss: 1.6328 - val_acc: 0.6607
Epoch 3/50
156/156 [==============================] - 345s 2s/step - loss: 0.0351 - acc: 0.9897 - val_loss: 1.3967 - val_acc: 0.7082
Epoch 4/50
156/156 [==============================] - 351s 2s/step - loss: 0.0308 - acc: 0.9906 - val_loss: 1.4681 - val_acc: 0.6956
Epoch 5/50
156/156 [==============================] - 354s 2s/step - loss: 0.0256 - acc: 0.9919 - val_loss: 1.6813 - val_acc: 0.6720
Epoch 6/50
156/156 [==============================] - 362s 2s/step - loss: 0.0220 - acc: 0.9931 - val_loss: 1.5749 - val_acc: 0.7252
Epoch 7/50
156/156 [==============================] - 348s 2s/step - loss: 0.0202 - acc: 0.9936 - val_loss: 1.8138 - val_acc: 0.6921
Epoch 8/50
156/156 [==============================] - 356s 2s/step - 

In [5]:
vgg16_model = load_model('vgg16_GAP_fc_11.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy']) 
    
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                  EarlyStopping(patience=10)])

vgg16_model.save('vgg16_GAP_fc_11_SDG.h5')

Epoch 1/50
156/156 [==============================] - 514s 3s/step - loss: 0.0166 - acc: 0.9953 - val_loss: 1.3666 - val_acc: 0.7222
Epoch 2/50
156/156 [==============================] - 357s 2s/step - loss: 0.0122 - acc: 0.9958 - val_loss: 1.1213 - val_acc: 0.7639
Epoch 3/50
156/156 [==============================] - 354s 2s/step - loss: 0.0092 - acc: 0.9970 - val_loss: 1.9253 - val_acc: 0.6755
Epoch 4/50
156/156 [==============================] - 350s 2s/step - loss: 0.0128 - acc: 0.9957 - val_loss: 1.5886 - val_acc: 0.7078
Epoch 5/50
156/156 [==============================] - 351s 2s/step - loss: 0.0087 - acc: 0.9967 - val_loss: 1.8673 - val_acc: 0.6995
Epoch 6/50
156/156 [==============================] - 352s 2s/step - loss: 0.0087 - acc: 0.9968 - val_loss: 1.3648 - val_acc: 0.7422
Epoch 7/50
156/156 [==============================] - 352s 2s/step - loss: 0.0074 - acc: 0.9977 - val_loss: 2.1056 - val_acc: 0.6590
Epoch 8/50
156/156 [==============================] - 348s 2s/step - 

In [7]:
# for layer in vgg16_model.layers[:7]:
#     layer.trainable = False
    
# for layer in vgg16_model.layers[7:]:
#     layer.trainable = True
    
# vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
# #                     optimizer=optimizers.Adam(),
#                     metrics=['accuracy']) 
    
# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=50,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
#                                    CSVLogger('vgg16_fine_tune'),
#                                   EarlyStopping(patience=10)])